# PD4 Jan Smoleń

In [52]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import dalex as dx
import pickle
np.random.seed = 46
import shap
import xgboost as xgb
from sklearn.metrics import accuracy_score
import warnings
warnings.filterwarnings('ignore')

In [53]:
wines_df = pd.read_csv('winequality-red.csv')
wines_df["is_good"]=wines_df.apply(lambda row: 1 if row.quality > 5 else 0, axis = 1)
X = wines_df.drop(columns = ['quality','is_good'])
y = wines_df[['is_good']]
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 1613)


## Przygotowanie modelu i explainera

In [54]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 1613)
xgb_model = xgb.XGBClassifier(objective = "binary:logistic", seed = 1613, min_child_weight=1, max_depth=10, 
                              learning_rate=0.05, gamma=0, colsample_bytree=0.4, use_label_encoder=False)
xgb_model.fit(X_train, y_train["is_good"])


[13:59:58] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.4, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.05, max_delta_step=0, max_depth=10,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=4, num_parallel_tree=1,
              random_state=1613, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
              seed=1613, subsample=1, tree_method='exact',
              use_label_encoder=False, validate_parameters=1, verbosity=None)

In [55]:
explainer = dx.Explainer(xgb_model, X_train, y_train,label="XGBoost")

Preparation of a new explainer is initiated

  -> data              : 1279 rows 11 cols
  -> target variable   : Parameter 'y' was a pandas.DataFrame. Converted to a numpy.ndarray.
  -> target variable   : 1279 values
  -> model_class       : xgboost.sklearn.XGBClassifier (default)
  -> label             : XGBoost
  -> predict function  : <function yhat_proba_default at 0x00000287063D44C0> will be used (default)
  -> predict function  : Accepts pandas.DataFrame and numpy.ndarray.
  -> predicted values  : min = 0.0238, mean = 0.537, max = 0.988
  -> model type        : classification will be used (default)
  -> residual function : difference between y and yhat (default)
  -> residuals         : min = -0.61, mean = -0.0004, max = 0.438
  -> model_info        : package xgboost

A new explainer has been created!


## 1

## XGBoost

In [56]:
tmp=accuracy_score(y_test['is_good'], xgb_model.predict(X_test))
print("Accuracy XGBoost = " + str(tmp))

Accuracy XGBoost = 0.815625


In [57]:
explainer.model_parts().plot()

Jak widać, prawie wszystkie zmienne mają wkład w predykcję naszego modelu. Jedyna, która się wyłamuje to zmienna `free sulfur dioxide` - chociaż jest pomijana przy tworzeniu wykresu dla wszystkich danych, to jak próbowałem spojrzeć tylko na nią, to jej ważność wynosiła praktycznie 0. 

In [58]:
explainer.model_parts(variables=["free sulfur dioxide"]).plot()

## 2

### SVM

In [59]:
from sklearn.svm import SVC
svm_model=SVC(C= 2000, gamma= 0.0006, kernel= 'rbf', random_state=42)

In [60]:
svm_model.fit(X_train, y_train)
explainer_svm = dx.Explainer(svm_model, X_train, y_train,label="SVM")

Preparation of a new explainer is initiated

  -> data              : 1279 rows 11 cols
  -> target variable   : Parameter 'y' was a pandas.DataFrame. Converted to a numpy.ndarray.
  -> target variable   : 1279 values
  -> model_class       : sklearn.svm._classes.SVC (default)
  -> label             : SVM
  -> predict function  : <function yhat_default at 0x00000287063D4430> will be used (default)
  -> predict function  : Accepts pandas.DataFrame and numpy.ndarray.
  -> predicted values  : min = 0.0, mean = 0.517, max = 1.0
  -> model type        : classification will be used (default)
  -> residual function : difference between y and yhat (default)
  -> residuals         : min = -1.0, mean = 0.0195, max = 1.0
  -> model_info        : package sklearn

A new explainer has been created!


In [61]:
tmp=accuracy_score(y_test['is_good'], svm_model.predict(X_test))
print("Accuracy SVM = " + str(tmp))

Accuracy SVM = 0.759375


In [62]:
explainer_svm.model_parts().plot()

## Random Forest

In [63]:
from sklearn.ensemble import RandomForestClassifier
rfc_model = RandomForestClassifier(random_state=1613, bootstrap= True,
 max_depth = 75,
 max_features='auto',
 min_samples_leaf= 2,
 min_samples_split= 2,
 n_estimators= 1050)
rfc_model.fit(X_train, y_train)




RandomForestClassifier(max_depth=75, min_samples_leaf=2, n_estimators=1050,
                       random_state=1613)

In [64]:
tmp=accuracy_score(y_test['is_good'], rfc_model.predict(X_test))
print("Accuracy RFC = " + str(tmp))

Accuracy RFC = 0.825


In [65]:
explainer_rfc =dx.Explainer(rfc_model, X_train, y_train,label="Random Forest")
explainer_rfc.model_parts().plot()

Preparation of a new explainer is initiated

  -> data              : 1279 rows 11 cols
  -> target variable   : Parameter 'y' was a pandas.DataFrame. Converted to a numpy.ndarray.
  -> target variable   : 1279 values
  -> model_class       : sklearn.ensemble._forest.RandomForestClassifier (default)
  -> label             : Random Forest
  -> predict function  : <function yhat_proba_default at 0x00000287063D44C0> will be used (default)
  -> predict function  : Accepts pandas.DataFrame and numpy.ndarray.
  -> predicted values  : min = 0.0114, mean = 0.537, max = 0.999
  -> model type        : classification will be used (default)
  -> residual function : difference between y and yhat (default)
  -> residuals         : min = -0.599, mean = -0.000248, max = 0.535
  -> model_info        : package sklearn

A new explainer has been created!


## 3

### Wnioski: 
* Wszystkie zmienne mają pozytywny wkład na predykcję modeli - w połączeniu z stosunkowo niewielką liczbą rekordów naszym zbiorze danych oznacza to, że słuszne wydaje się pozostawienie wszystkich kolumn 
* Jeżeli chodzi o proporcjonalną ważność zmiennych, to **XGBoost** i **Random Forest** są dosyć podobne i odpowiadają mniej więcej korelacjom ze zmienną celu
* W **SVM** za to nieoczekiwanie duże znaczenie mają zmienne dotyczące tlenku siarki - 'total ...' i 'free sulfur dioxide'. Myśle że wynika to z faktu, że nie wystandaryzowaliśmy naszych zmiennych, a te dwie kolumny przyjmują stosunkowo duże wielkośći. Sprawdźmy!

In [66]:
X_stand=(X-X.mean())/X.std()
X_train, X_test, y_train, y_test = train_test_split(X_stand, y, test_size = 0.2, random_state = 1613)
svm_model2=SVC()
svm_model2.fit(X_train, y_train)
explainer_svm2 = dx.Explainer(svm_model, X_train, y_train,label="SVM")


Preparation of a new explainer is initiated

  -> data              : 1279 rows 11 cols
  -> target variable   : Parameter 'y' was a pandas.DataFrame. Converted to a numpy.ndarray.
  -> target variable   : 1279 values
  -> model_class       : sklearn.svm._classes.SVC (default)
  -> label             : SVM
  -> predict function  : <function yhat_default at 0x00000287063D4430> will be used (default)
  -> predict function  : Accepts pandas.DataFrame and numpy.ndarray.
  -> predicted values  : min = 0.0, mean = 0.0414, max = 1.0
  -> model type        : classification will be used (default)
  -> residual function : difference between y and yhat (default)
  -> residuals         : min = -1.0, mean = 0.495, max = 1.0
  -> model_info        : package sklearn

A new explainer has been created!


In [73]:
explainer_svm2.model_parts().plot()

Po wystandaryzowaniu zmiennych, nasz wykres wygląda kompletnie inaczej. Najważniejszą rolę odgrywają teraz zmienne dotyczące kwasowości. Co ciekawe, pojawiły się także zmienne nie wnoszące nic do predykcji, co jest nowością. Jednak wynik **SVM** jest znaczenie gorszy od **XGBoosta** i **Random Forest**, więc nie będziemy się tym bardzo sugerować.  